In [9]:
import sys
import soundcloud

In [5]:
from functools import partial
from utils import get_results, handle_http_errors

In [15]:
def getUserAttr(resource, attr):
#   if hasattr(resource, 'user'): return resource.user[attr]
    if hasattr(resource, attr): return getattr(resource, attr)
    return None

getUsername = partial(getUserAttr, attr='username')
getUserid = partial(getUserAttr, attr='id')

In [10]:
# A global artist graph used to iterate through the various algorithms.
# Each node is artist id, with edges weighted by activity between then.
#profileGraph = nx.MultiDiGraph()

client = soundcloud.Client(client_id='454aeaee30d3533d6d8f448556b50f23')

name = input("Enter a soundcloud artist to analyze: ")


Enter a soundcloud artist to analyze: sybyr


In [11]:
# Artist of interest
search = client.get('/users/', q = name)[0]

In [12]:
search

In [13]:
search.__dict__

{'obj': {'avatar_url': 'https://i1.sndcdn.com/avatars-cIS3ELGYY7j2e0q9-OEySHA-large.jpg',
  'id': 13377746,
  'kind': 'user',
  'permalink_url': 'http://soundcloud.com/sybyr',
  'uri': 'https://api.soundcloud.com/users/13377746',
  'username': 'Sybyr',
  'permalink': 'sybyr',
  'last_modified': '2020/07/17 04:43:29 +0000',
  'first_name': 'Sybyrmusic.com',
  'last_name': '',
  'full_name': 'Sybyrmusic.com',
  'city': '',
  'description': 'bookings/email: sybyrcentral@gmail.com\nAnti-World: http://antiworld.co (@antiworldglobe) on all networks\n(Prod. by Landfill) at http://prodbylandfill.com',
  'country': 'United States',
  'track_count': 436,
  'public_favorites_count': 0,
  'followers_count': 51690,
  'followings_count': 22,
  'plan': 'Pro Unlimited',
  'myspace_name': None,
  'discogs_name': None,
  'website_title': 'click Here',
  'website': 'http://antiworld.lnk.to/SYBYR',
  'reposts_count': 99,
  'comments_count': 110,
  'online': False,
  'likes_count': 0,
  'playlist_count': 3

In [22]:
print("Artist interpreted as: %s" % getUsername(search))

Artist interpreted as: Sybyr


In [23]:
@handle_http_errors
def id2username(profile, kind='users'):
    global id2username_dict
    username = id2username_cache.get(profile, None)
    if username is not None: return username

    # username is none, we don't have it in cache
    result = client.get('/%s/%s' % (kind, str(profile)))
    if kind == 'comments':
        username = result.user['username']
    elif kind == 'tracks':
        username = result.title
    else:
        username = result.username
    # encode it correctly
    username = str(username.encode('utf-8'))
    id2username_cache[profile] = username
    return username

In [24]:
# list of profiles to query
profiles_to_query = [search.id]

In [25]:
profiles_to_query

[13377746]

In [26]:
depth = 2
i = 0

# list of profiles we could not query
unavailable_profiles = []

In [30]:
id2username_cache = {}

In [31]:
for t in range(depth):

    print("Iteration " + str(t))

    profiles_to_enqueue = list(set(profiles_to_enqueue))

    for profile in profiles_to_enqueue:
        username = id2username(profile)
        if username:
            print("\t", "Enqueueing: %s (%s)" % (username, profile))

Iteration 0
	 Enqueueing: b'Sybyr' (13377746)
Iteration 1
	 Enqueueing: b'Sybyr' (13377746)
